In [1]:
# Benötigte Bibliotheken und Module
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
import requests
import re
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.ticker as mtick

In [2]:
# CSV-Datei einlesen
file_path_persons = '../../data/clean/Data_Masterarbeit - list_named_after_humans.csv'
file_path_all_streets = '../../data/clean/Data_Masterarbeit - list_complete.csv'
file_path_street_layout = "../../data/raw/lhd_strassenzuege.csv"

df_persons = pd.read_csv(file_path_persons, sep=",")
df_all = pd.read_csv(file_path_all_streets, sep=",")
df_streets = pd.read_csv(file_path_street_layout, sep=";")

## Für Datei "Data_Masterarbeit - list complete"

### Vorbereitungen

In [3]:
# 1. Lokale Straßendaten laden (df_streets)

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Tätigkeitsfeld-Daten laden (df_all)

df_all["strasse_key"] = df_all["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Tätigkeitsfeld
strassen_merge = strassen_gdf.merge(
    df_all[["strasse_key", "Benennungstyp", "Grundwort", "lokaler Bezug"]], # Nur die benötigten Spalten aus df_all
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)


# 3. Gemarkungen laden (GeoJSON)
gemarkungen_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadt_json = requests.get(gemarkungen_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    print("Gemarkungen erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")



# 4. Räumliche Verknüpfung Straße ↔ Gemarkung
# WICHTIG: Sicherstellen, dass beide GeoDataFrames ein gültiges CRS haben
if strassen_merge.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_merge...")
    strassen_merge = strassen_merge.to_crs(stadt_gdf.crs)

strassen_gemarkung = gpd.sjoin(strassen_merge, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Alle einzigartigen Benennungstypen finden
all_types = strassen_gemarkung["Benennungstyp"].dropna().unique()
print(f"Gefundene Benennungstypen zum Iterieren: {all_types}")

# Gesamtanzahl aller nach Personen/Typen benannter Straßen pro Gemarkung (Nenner für die Prozentrechnung)
personen_strassen = strassen_gemarkung.dropna(subset=["Benennungstyp"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_key"])

Gemarkungen erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.
Gefundene Benennungstypen zum Iterieren: ['Person real' 'Ort' 'Region' 'Bevölkerungsgruppe' 'Bauwerk/Infrastruktur'
 'Sonstige' 'Selbstreferenziell' 'Natur' 'Gegenstand' 'weitere Geografika'
 'Gewerbe/Industrie' 'Gruppe real' 'Beruf/Funktion' 'Abstrakta' 'Ereignis'
 'Person fiktiv' 'Unbekannt' 'Siedlung' 'Burg' 'Land' 'Kunstwerk']


### Abfrage

In [4]:

# 1. Absolute Anzahl pro Gemarkung & Benennungstyp
counts = personen_strassen_unique.groupby(["gem_nam", "lokaler Bezug"]).size().unstack(fill_value=0)

# Prozentualer Anteil
anteile = counts.div(counts.sum(axis=1), axis=0)

# 2. Gemarkungen filtern
gewuenschte_gemarkungen = [
   "Gorbitz",
   "Prohlis"
]  # <-- HIER Gemarkungen anpassen!

vorhandene_gemarkungen = [gem for gem in gewuenschte_gemarkungen if gem in counts.index]

counts_gefiltert = counts.loc[vorhandene_gemarkungen].copy()
anteile_gefiltert = anteile.loc[vorhandene_gemarkungen].copy()

# Finde Spalten (Kategorien), deren Summe in den gefilterten Gemarkungen 0 ist
spalten_summe = counts_gefiltert.sum(axis=0)
relevante_spalten = spalten_summe[spalten_summe > 0].index

# Wende diesen Filter auf beide DataFrames an
counts_gefiltert = counts_gefiltert[relevante_spalten]
anteile_gefiltert = anteile_gefiltert[relevante_spalten]

# 3. Gestapeltes Balkendiagramm
fig, ax = plt.subplots(figsize=(12, 6))

# Farbzuordnung für die Benennungstypen
farben = cm.tab20.colors + cm.tab20b.colors
fields = list(anteile_gefiltert.columns)   # natürliche Reihenfolge
farbe_map = {field: mcolors.to_hex(farben[i % len(farben)]) for i, field in enumerate(fields)}

x = np.arange(len(anteile_gefiltert.index))
width = 0.8

# Balken plotten (bottom → top)
bottom = np.zeros(len(x))
bars = []
for cat in fields:
    heights = anteile_gefiltert[cat].values
    bar = ax.bar(x, heights, width, bottom=bottom, label=cat, color=farbe_map[cat])
    bars.append(bar)
    bottom = bottom + heights

MIN_HEIGHT_FOR_PERCENTAGE = 0.08  # 8% Anteil für die Anzeige der Prozentzahl
MIN_HEIGHT_FOR_ABSOLUTE = 0.03    # 3% Anteil für die Anzeige der absoluten Zahl

# Absolute Zahlen in die Segmente schreiben
for bar_group, cat in zip(bars, fields):
    abs_vals = counts_gefiltert[cat].values
    perc_vals = anteile_gefiltert[cat].values * 100
    for rect, abs_val, perc_val in zip(bar_group, abs_vals, perc_vals):
        h = rect.get_height()
        if h > 0 and abs_val > 0:
            
            if h >= MIN_HEIGHT_FOR_PERCENTAGE:
                # Genug Platz für absolute Zahl UND Prozente
                text_color = "white" if h > 0.08 else "black" # Behält ursprüngliche Farb-Logik bei
                text_label = f"{int(abs_val)}\n({perc_val:.1f}%)"
            elif h >= MIN_HEIGHT_FOR_ABSOLUTE:
                # Nur Platz für die absolute Zahl
                text_color = "black" 
                text_label = f"{int(abs_val)}"
            else:
                # Zu wenig Platz, nichts anzeigen
                continue
            ax.text(
                rect.get_x() + rect.get_width() / 2,
                rect.get_y() + h / 2,
                text_label,
                ha="center", va="center", fontsize=12, color=text_color
            )

# Gesamtsummen oben drauf
totals = counts_gefiltert.sum(axis=1).values
for xi, tot in zip(x, totals):
    ax.text(xi, 1.02, str(int(tot)), ha="center", va="bottom", fontweight="bold", fontsize=12)

# Achsen & Titel
ax.set_xticks(x)
ax.set_xticklabels(anteile_gefiltert.index, rotation=45, ha="right", fontsize=12)
ax.set_ylabel("Anteil der Straßen")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
ax.set_xlabel("Gemarkung")
ax.set_title("Verteilung lokaler Bezug pro Gemarkung")
ax.set_ylim(0, 1.1)
# Legende (umgedreht, damit Reihenfolge mit der visuellen Stapelung übereinstimmt)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="lokaler Bezug", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=12)

plt.tight_layout()
plt.savefig("../../results/charts/bar_charts_district_comparison/lokaler_Bezug_Großwohnsiedlungen.png", dpi=300)
plt.close()

## Für Datei "Data_Masterarbeit - list named after humans"

### Vorbereitung

In [6]:
# 1. Lokale Straßendaten laden (df_streets)

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Tätigkeitsfeld-Daten laden (df_all)

df_persons["strasse_key"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Tätigkeitsfeld
strassen_merge = strassen_gdf.merge(
    df_persons[["strasse_key", "Tätigkeitsfeld", "Tätigkeitsfeld II", "Wirkungsbereich", "Partei"]], # Nur die benötigten Spalten aus df_all
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)


# 3. Gemarkungen laden (GeoJSON)
gemarkungen_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadt_json = requests.get(gemarkungen_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    print("Gemarkungen erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")



# 4. Räumliche Verknüpfung Straße ↔ Gemarkung
# WICHTIG: Sicherstellen, dass beide GeoDataFrames ein gültiges CRS haben
if strassen_merge.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht übereinu...")
    strassen_merge = strassen_merge.to_crs(stadt_gdf.crs)

strassen_gemarkung = gpd.sjoin(strassen_merge, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Alle einzigartigen Tätigkeitsfeld finden
all_types = strassen_gemarkung["Tätigkeitsfeld"].dropna().unique()
print(f"Gefundene Tätigkeitsfelder zum Iterieren: {all_types}")

# Gesamtanzahl aller nach Personen/Typen benannter Straßen pro Gemarkung (Nenner für die Prozentrechnung)
personen_strassen = strassen_gemarkung.dropna(subset=["Tätigkeitsfeld"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_key"])

Gemarkungen erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.
Gefundene Tätigkeitsfelder zum Iterieren: ['politisch' 'religiös' 'kulturell' 'wissenschaftlich' 'ökonomisch'
 'militärisch' 'administrativ']


### Abfrage

In [15]:

# 1. Absolute Anzahl pro Gemarkung & Benennungstyp
counts = personen_strassen_unique.groupby(["gem_nam", "Wirkungsbereich"]).size().unstack(fill_value=0)

# Prozentualer Anteil
anteile = counts.div(counts.sum(axis=1), axis=0)

# 2. Gemarkungen filtern
gewuenschte_gemarkungen = [
    "Pieschen",
    "Löbtau",
    "Blasewitz",
    "Loschwitz"
]  # <-- HIER Gemarkungen anpassen!

vorhandene_gemarkungen = [gem for gem in gewuenschte_gemarkungen if gem in counts.index]

counts_gefiltert = counts.loc[vorhandene_gemarkungen].copy()
anteile_gefiltert = anteile.loc[vorhandene_gemarkungen].copy()

# Finde Spalten (Kategorien), deren Summe in den gefilterten Gemarkungen 0 ist
spalten_summe = counts_gefiltert.sum(axis=0)
relevante_spalten = spalten_summe[spalten_summe > 0].index

# Wende diesen Filter auf beide DataFrames an
counts_gefiltert = counts_gefiltert[relevante_spalten]
anteile_gefiltert = anteile_gefiltert[relevante_spalten]

# 3. Gestapeltes Balkendiagramm
fig, ax = plt.subplots(figsize=(12, 6))

# Farbzuordnung für die Benennungstypen
farben = cm.tab20.colors + cm.tab20b.colors
fields = list(anteile_gefiltert.columns)   # natürliche Reihenfolge
farbe_map = {field: mcolors.to_hex(farben[i % len(farben)]) for i, field in enumerate(fields)}

x = np.arange(len(anteile_gefiltert.index))
width = 0.8

# Balken plotten (bottom → top)
bottom = np.zeros(len(x))
bars = []
for cat in fields:
    heights = anteile_gefiltert[cat].values
    bar = ax.bar(x, heights, width, bottom=bottom, label=cat, color=farbe_map[cat])
    bars.append(bar)
    bottom = bottom + heights

# Absolute Zahlen in die Segmente schreiben
for bar_group, cat in zip(bars, fields):
    abs_vals = counts_gefiltert[cat].values
    perc_vals = anteile_gefiltert[cat].values * 100
    for rect, abs_val, perc_val in zip(bar_group, abs_vals, perc_vals):
        h = rect.get_height()
        if h > 0 and abs_val > 0:
            text_color = "white" if h > 0.08 else "black"
            text_label = f"{int(abs_val)}\n({perc_val:.1f}%)"
            ax.text(
                rect.get_x() + rect.get_width() / 2,
                rect.get_y() + h / 2,
                text_label,
                ha="center", va="center", fontsize=10, color=text_color
            )

# Gesamtsummen oben drauf
totals = counts_gefiltert.sum(axis=1).values
for xi, tot in zip(x, totals):
    ax.text(xi, 1.02, str(int(tot)), ha="center", va="bottom", fontweight="bold", fontsize=12)

# Achsen & Titel
ax.set_xticks(x)
ax.set_xticklabels(anteile_gefiltert.index, rotation=45, ha="right", fontsize=12)
ax.set_ylabel("Anteil der Straßen")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
ax.set_xlabel("Gemarkung")
ax.set_title("Verteilung nach Wirkungsbereich pro Gemarkung")
ax.set_ylim(0, 1.1)
# Legende (umgedreht, damit Reihenfolge mit der visuellen Stapelung übereinstimmt)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="Wirkungsbereich", bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=12)

plt.tight_layout()
plt.savefig("../../results/charts/bar_charts_district_comparison/Wirkungsbereich_Arbeiter_und_Bürgerviertel.png", dpi=300)
plt.close()

### Ausgabe der Straßennamen

In [26]:
SPALTE_MIT_STRASSENNAMEN = 'strasse' # <--- BITTE ANPASSEN

# 1. Wir verwenden dieselben Gemarkungen, die du im Plot filtern willst
gewuenschte_gemarkungen = [
    "Altstadt I"
]

# 2. Filtere den ursprünglichen DataFrame auf diese Gemarkungen
df_gefiltert = personen_strassen_unique[
    personen_strassen_unique['gem_nam'].isin(gewuenschte_gemarkungen)
]

# 3. Gruppiere neu und sammle die Straßennamen in Listen
strassen_listen = df_gefiltert.groupby(['gem_nam', 'Benennungstyp'])[SPALTE_MIT_STRASSENNAMEN].apply(list)

# 1. Zeigt alle Zeilen an (z.B. alle Gemarkung/Tätigkeitsfeld-Kombinationen)
pd.set_option('display.max_rows', None)

# 2. Zeigt den VOLLEM Inhalt jeder Zelle an (KEINE "..." in der Liste)
pd.set_option('display.max_colwidth', None)

# 4. Ausgabe
print("--- Straßennamen pro Gemarkung und Tätigkeitsfeld ---")
strassen_listen

--- Straßennamen pro Gemarkung und Tätigkeitsfeld ---


gem_nam     Benennungstyp        
Altstadt I  Abstrakta                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             [Trabantengasse]
            Bauwerk/Infrastruktur                                                  

### Sonderfall Partei

In [7]:
# 1. Daten-Transformation: Parteien-Strings aufteilen und expandieren

# Kopiere die relevanten Spalten (Gemarkung und Partei)
df_expanded = personen_strassen_unique[['gem_nam', 'Partei']].copy()

# Entferne Zeilen, bei denen die Partei nicht bekannt ist (NaN)
df_expanded = df_expanded.dropna(subset=['Partei'])

# Teile die 'Partei'-Spalte an Kommas, um Listen zu erstellen
df_expanded['Partei_list'] = df_expanded['Partei'].str.split(',')

# 'explode()' erstellt für jedes Element in der Liste eine neue Zeile
df_expanded = df_expanded.explode('Partei_list')

# Bereinige Leerzeichen
df_expanded['Partei'] = df_expanded['Partei_list'].str.strip()

# ----- KORREKTUR HIER -----
# 1. Absolute Anzahl pro Gemarkung & Partei
# Wir MÜSSEN das transformierte DataFrame 'df_expanded' verwenden!
counts = df_expanded.groupby(["gem_nam", "Partei"]).size().unstack(fill_value=0)
# -------------------------

# Prozentualer Anteil
anteile = counts.div(counts.sum(axis=1), axis=0)


# 2. Gemarkungen filtern
gewuenschte_gemarkungen = [
    "Gorbitz",
    "Prohlis"
]  # <-- HIER Gemarkungen anpassen!

vorhandene_gemarkungen = [gem for gem in gewuenschte_gemarkungen if gem in counts.index]

counts_gefiltert = counts.loc[vorhandene_gemarkungen].copy()
anteile_gefiltert = anteile.loc[vorhandene_gemarkungen].copy()

# Finde Spalten (Kategorien), deren Summe in den gefilterten Gemarkungen 0 ist
spalten_summe = counts_gefiltert.sum(axis=0)
relevante_spalten = spalten_summe[spalten_summe > 0].index

# Wende diesen Filter auf beide DataFrames an
counts_gefiltert = counts_gefiltert[relevante_spalten]
anteile_gefiltert = anteile_gefiltert[relevante_spalten]

# 3. Gestapeltes Balkendiagramm
fig, ax = plt.subplots(figsize=(12, 6))

# Farbzuordnung für die Benennungstypen
farben = cm.tab20.colors + cm.tab20b.colors
fields = list(anteile_gefiltert.columns)   # natürliche Reihenfolge
farbe_map = {field: mcolors.to_hex(farben[i % len(farben)]) for i, field in enumerate(fields)}

x = np.arange(len(anteile_gefiltert.index))
width = 0.8

# Balken plotten (bottom → top)
bottom = np.zeros(len(x))
bars = []
for cat in fields:
    heights = anteile_gefiltert[cat].values
    bar = ax.bar(x, heights, width, bottom=bottom, label=cat, color=farbe_map[cat])
    bars.append(bar)
    bottom = bottom + heights

# Absolute Zahlen in die Segmente schreiben
for bar_group, cat in zip(bars, fields):
    abs_vals = counts_gefiltert[cat].values
    perc_vals = anteile_gefiltert[cat].values * 100
    for rect, abs_val, perc_val in zip(bar_group, abs_vals, perc_vals):
        h = rect.get_height()
        if h > 0 and abs_val > 0:
            text_label = f"{int(abs_val)}\n({perc_val:.1f}%)"
            text_color = "white" if h > 0.08 else "black"
            ax.text(
                rect.get_x() + rect.get_width() / 2,
                rect.get_y() + h / 2,
                text_label,
                ha="center", va="center", fontsize=10, color=text_color
            )

# Gesamtsummen oben drauf
totals = counts_gefiltert.sum(axis=1).values
for xi, tot in zip(x, totals):
    ax.text(xi, 1.02, str(int(tot)), ha="center", va="bottom", fontweight="bold", fontsize=12)

# Achsen & Titel
ax.set_xticks(x)
ax.set_xticklabels(anteile_gefiltert.index, rotation=45, ha="right")
ax.set_ylabel("Anteil der Straßen")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
ax.set_xlabel("Gemarkung")
ax.set_title("Prozentuale Anteile der Tätigkeitsfeld pro Gemarkung (Gefiltert)")
ax.set_ylim(0, 1.1)

# Legende (umgedreht, damit Reihenfolge mit der visuellen Stapelung übereinstimmt)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="Tätigkeitsfeld", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
#plt.savefig("../../results/charts/bar_charts/Balkendiagramm_prozentuale_Anteile_Parteien_Arbeiter_und_Bürgerviertel.png", dpi=300)
plt.close()

### Vergleich Gemarkungen Tätigkeitsfeld II

In [ ]:
# Liste der zu analysierenden Tätigkeitsfelder
taetigkeitsfelder = [
    "kulturell"
]

# Gewünschte Gemarkungen
gewuenschte_gemarkungen = [
    "Pieschen",
    "Löbtau",
    "Blasewitz",
    "Striesen"
]

# Die gesamte Logik wird nun für jedes Tätigkeitsfeld ausgeführt
for taetigkeitsfeld in taetigkeitsfelder:
    
    print(f"Starte Verarbeitung für Tätigkeitsfeld: '{taetigkeitsfeld}'")
    
    
    # SCHRITT 0: DataFrame filtern, um nur Einträge des aktuellen Feldes zu behalten
    # Annahme: 'personen_strassen_unique' ist der Quelldatenrahmen.
    df_filtered = personen_strassen_unique[personen_strassen_unique['Tätigkeitsfeld'] == taetigkeitsfeld].copy()

    # Nur fortfahren, wenn Daten für das Tätigkeitsfeld vorhanden sind
    if df_filtered.empty:
        print(f"Keine Einträge für das Tätigkeitsfeld: '{taetigkeitsfeld}'. Überspringe Plot.")
        continue

    # 1. Absolute Anzahl pro Gemarkung & Tätigkeitsfeld II zählen
    counts = df_filtered.groupby(["gem_nam", "Tätigkeitsfeld II"]).size().unstack(fill_value=0)

    # Prozentualer Anteil
    anteile = counts.div(counts.sum(axis=1), axis=0)

    # 2. Gemarkungen filtern
    vorhandene_gemarkungen = [gem for gem in gewuenschte_gemarkungen if gem in counts.index]
    
    if not vorhandene_gemarkungen:
        print(f"Keine der gewünschten Gemarkungen ({', '.join(gewuenschte_gemarkungen)}) hat Einträge für '{taetigkeitsfeld}'. Überspringe Plot.")
        continue

    counts_gefiltert = counts.loc[vorhandene_gemarkungen].copy()
    anteile_gefiltert = anteile.loc[vorhandene_gemarkungen].copy()
    
    # Bestimme die Felder, die tatsächlich in den gefilterten Daten VORKOMMEN
    # Diese Logik (die du bereits hattest) stellt sicher, dass nur Spalten > 0 berücksichtigt werden.
    fields_to_plot = list(anteile_gefiltert.columns[anteile_gefiltert.sum(axis=0) > 0])
    
    # Wenn keine Unterkategorien vorhanden sind, überspringen
    if not fields_to_plot:
        print(f"Keine relevanten Unterkategorien ('Tätigkeitsfeld II') für '{taetigkeitsfeld}' in den gefilterten Gemarkungen gefunden. Überspringe Plot.")
        continue

    # 3. Gestapeltes Balkendiagramm
    fig, ax = plt.subplots(figsize=(12, 6))

    # KORREKTUR 1: Erweiterte Farbpalette (tab20 + tab20b) für 40 Farben
    farben = cm.tab20.colors + cm.tab20b.colors
    
    # Deine Logik für konsistente Farben (sehr gut!):
    all_fields = list(counts.columns) # Alle existierenden Unterkategorien
    farbe_map = {field: mcolors.to_hex(farben[i % len(farben)]) for i, field in enumerate(all_fields)}

    x = np.arange(len(anteile_gefiltert.index))
    width = 0.8

    # Balken plotten (bottom → top)
    bottom = np.zeros(len(x))
    bars = []
    
    # Iteriere nur über die tatsächlich zu plottenden Felder (fields_to_plot)
    for cat in fields_to_plot:
        heights = anteile_gefiltert[cat].values
        bar = ax.bar(x, heights, width, bottom=bottom, label=cat, color=farbe_map[cat])
        bars.append(bar)
        bottom = bottom + heights

    # Absolute Zahlen UND Prozentuale Anteile in die Segmente schreiben
    for bar_group, cat in zip(bars, fields_to_plot):
        abs_vals = counts_gefiltert[cat].values
        perc_vals = anteile_gefiltert[cat].values
        
        for rect, abs_val, perc_val in zip(bar_group, abs_vals, perc_vals):
            h = rect.get_height()
            
            if h > 0 and abs_val > 0:
                text_color = "white" if h > 0.08 else "black"
                
                # Formatierung des Texts: Absolute Zahl (Prozent)
                text = f"{int(abs_val)} ({perc_val:.1%})"
                
                ax.text(
                    rect.get_x() + rect.get_width() / 2,
                    rect.get_y() + h / 2,
                    text,
                    ha="center", va="center", fontsize=7, color=text_color,
                    fontweight="bold" if h > 0.15 else "normal"
                )

    # Gesamtsummen oben drauf 
    totals = counts_gefiltert.sum(axis=1).values
    for xi, tot in zip(x, totals):
        ax.text(xi, 1.02, str(int(tot)), ha="center", va="bottom", fontweight="bold")

    # Achsen & Titel
    ax.set_xticks(x)
    ax.set_xticklabels(anteile_gefiltert.index, rotation=45, ha="right")
    ax.set_ylabel(f"Anteil der {taetigkeitsfeld}en Straßenbenennungen")
    
    # KORREKTUR 2: Y-Achse als Prozent formatieren (z.B. "80%" statt "0.8")
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    
    ax.set_xlabel("Gemarkung")
    ax.set_title(f"Prozentuale Anteile der Unterkategorien von '{taetigkeitsfeld}' pro Gemarkung (Absolutzahl & Anteil)")

    # Legende (umgedreht, zeigt nun NUR die tatsächlich vorkommenden Unterkategorien)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], title=f"Tätigkeitsfeld II (Unterkategorie von '{taetigkeitsfeld}')", bbox_to_anchor=(1.05, 1), loc="upper left")

    plt.tight_layout()
    
    # Dynamischer Dateiname
    filename = f"../../results/charts/bar_charts/Balkendiagramm_prozentuale_Anteile_Gemarkungen_{taetigkeitsfeld}_Unterteilung.png"
    plt.savefig(filename, dpi=300)
    plt.close(fig)

Starte Verarbeitung für Tätigkeitsfeld: 'kulturell'
